In [17]:
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
import eli5
import nltk
nltk.download('punkt')
from IPython.display import display
import numpy as np
from sklearn import linear_model
import pandas as pd


import features
import config
import data_loader
import os

seed_value = 42  # random seed of 42 for all experiments
os.environ['PYTHONHASHSEED'] = str(seed_value)
np.random.seed(seed_value)

[nltk_data] Downloading package punkt to /home/max/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def train_tfidf(train):
    vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', encoding='utf-8')
    train_sentences, test_sentences = [], []
    for x in train:
        train_sentences.append(features.get_words_str(x))
    X_train = vectorizer.fit_transform(train_sentences)
    return vectorizer, X_train


def make_predictions(piper_files, annotated_files, vectorizer, model):
    count = 0
    test_sentences = []
    filenames = []

    rootdir = '../../dataset/minNarrative_txtfiles/'
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if file not in piper_files: # Only look at non-annotated files
                filepath = os.path.join(subdir, file)
                filenames.append(filepath)
                count += 1
                with open(filepath) as file:
                    text = file.read().strip()
                test_sentences.append(' '.join(features.filter_punct(word_tokenize(text))))
    X_test = vectorizer.transform(test_sentences)

    print("Making predictions for {} documents".format(count))
#     preds_with_probs = model.predict_proba(X_test)[:, 1]
    preds = model.predict(X_test.toarray())
    preds[preds > 1.0] = 1.0
    preds[preds < 0.0] = 0
    return preds, filenames, X_test


def SVM_model(X_train, Y):
    tuned_parameters = [{'C': [1], 'kernel': ['linear']}, ]
    algo = SVC(probability=True)

    param_object = ParameterGrid(tuned_parameters)
    for param_dict in param_object:
        print("Running for parameters:", param_dict)
        algo.set_params(**param_dict)  # set the desired hyperparameters

        print("Training SVM TFIDF model with {} documents".format(Y.shape[0]))
        clf = algo.fit(X_train, Y)

        return clf

    
def Ridge_model(X_train, Y):
    tuned_parameters = [{}]
    algo = linear_model.Ridge()

    param_object = ParameterGrid(tuned_parameters)
    for param_dict in param_object:
        print("Running for parameters:", param_dict)
        algo.set_params(**param_dict)  # set the desired hyperparameters

        print("Training SVM TFIDF model with {} documents".format(Y.shape[0]))
        clf = algo.fit(X_train, Y)

        return clf

def TheilSenRegressor_model(X_train, Y):
    tuned_parameters = [{}]
    algo = linear_model.TheilSenRegressor(random_state=0, n_jobs=-1)
    
    param_object = ParameterGrid(tuned_parameters)
    for param_dict in param_object:
        print("Running for parameters:", param_dict)
        algo.set_params(**param_dict)  # set the desired hyperparameters

        print("Training TheilSenRegressor TFIDF model with {} documents".format(len(Y)))
        clf = algo.fit(X_train, Y)

        return clf

    
def average_probabilities(preds_with_probs, filenames):
    genre_probabilities = dict()
    for i in range(len(filenames)):
        genre = filenames[i].split('/')[-1].split('_')[0]
        if genre == 'FOX' or genre == 'CNN':
            genre = 'OPINION'

        if genre in genre_probabilities:
            genre_probabilities[genre].append(preds_with_probs[i])
        else:
            genre_probabilities[genre] = [preds_with_probs[i]]

    for k, v in genre_probabilities.items():
        avg = round(sum(v) / len(v), 3)
        print('Genre {}, {} documents, average predicted probability: {}'.format(k, len(v), avg))


In [3]:
# Loads annotated filenames and labels (334 files)
X, Y = data_loader.load_annotated_data(threshold=2.5)

# Change to MinNarrative_ReaderData_Final to make predictions for data of Piper and Bagga (2022)
annotated_df = pd.read_csv('../../dataset/Universal_Annotation_Results_Selection.csv')

piper_df = pd.read_csv('../../dataset/MinNarrative_ReaderData_Final.csv')
piper_files = piper_df[['FILENAME']].values
map_fname_score = dict(annotated_df[['FILENAME', 'avg_overall']].values)

# Rescale Reader Annotation Scores from 1 to 5, to 0 to 1.
Y = [((1 - 0) * (map_fname_score[fname] - 1) / (5 - 1)) for fname in X]


# Creates Tfidf vectorizer using the annotated files
vectorizer, X_train = train_tfidf(X)

# # Trains SVM model using the annotated files
# svm = SVM_model(X_train, Y)
model = TheilSenRegressor_model(X_train.toarray(), Y)


# Makes predictions on the rest of the data set (17k+ documents) using the SVM model and Tfidf Vectorizer
# preds_with_probs, filenames, X_test = make_predictions(X, vectorizer, svm)


preds_with_probs, filenames, X_test = make_predictions(piper_files, X, vectorizer, model)

Loading annotated data from: ../../dataset/Universal_Annotation_Results_Selection.csv
Running for parameters: {}
Training TheilSenRegressor TFIDF model with 325 documents
Making predictions for 17372 documents


In [4]:
print(len(filenames))

17372


In [5]:
import pickle

In [6]:
with open('PredsUniversalsTFIDF_TSR_17372docs.pickle', 'wb') as f:
    pickle.dump(preds_with_probs, f)


# Average predicted probability per genre

In [6]:
average_probabilities(preds_with_probs, filenames)

Genre SHORT, 488 documents, average predicted probability: 0.559
Genre OPINION, 1611 documents, average predicted probability: 0.338
Genre NOVEL19C, 1025 documents, average predicted probability: 0.502
Genre FLASH, 877 documents, average predicted probability: 0.535
Genre LEGAL, 1084 documents, average predicted probability: 0.06
Genre MIXED, 1000 documents, average predicted probability: 0.397
Genre BIO, 973 documents, average predicted probability: 0.426
Genre ACADEMIC-LITSTUDY, 528 documents, average predicted probability: 0.257
Genre ACADEMIC-SCIENCE, 967 documents, average predicted probability: 0.133
Genre APHORISM, 467 documents, average predicted probability: 0.271
Genre NOVEL-CONT, 951 documents, average predicted probability: 0.562
Genre FABLE, 263 documents, average predicted probability: 0.51
Genre ROC, 977 documents, average predicted probability: 0.502
Genre HIST, 1050 documents, average predicted probability: 0.372
Genre SCOTUS, 965 documents, average predicted probabili

# Feature Analysis

In [7]:
# Get corresponding feature names of Tfidf Vectorizer
feature_names = list(vectorizer.get_feature_names_out())

In [10]:
# Get index of passages with lowest and highest predicted degree of narrativty
highest_degree = np.where(preds_with_probs ==np.amax(preds_with_probs))[0][0]
lowest_degree = np.where(preds_with_probs ==np.amin(preds_with_probs))[0][0]
print(filenames[highest_degree])
print(filenames[lowest_degree])

../../dataset/minNarrative_txtfiles/reddit-stories/REDDIT_5S_bhbyx-143-story.txt
../../dataset/minNarrative_txtfiles/legal-contracts/LEGAL_5S_1_ExactSciencesCorp_20180822_8-K_EX-10-1_11331629_EX-10-1_Promotion-Agreement.txt


In [11]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [12]:
def show_weights(fname):
    with open(fname) as file:
        text = file.read().strip()
        text = ' '.join(word_tokenize(text))
        display(eli5.explain_prediction(model, text, vec=vectorizer, feature_names=feature_names))

print(preds_with_probs[lowest_degree])
print(filenames[lowest_degree])
show_weights(filenames[lowest_degree])

0.0
../../dataset/minNarrative_txtfiles/legal-contracts/LEGAL_5S_1_ExactSciencesCorp_20180822_8-K_EX-10-1_11331629_EX-10-1_Promotion-Agreement.txt


Explanation(estimator='TheilSenRegressor(n_jobs=-1, random_state=0)', description=None, error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=0.3039374326740319, std=None, value=1.0), FeatureWeight(feature='the', weight=0.010295544890209084, std=None, value=0.15370762276447825), FeatureWeight(feature='at', weight=0.005059551380042976, std=None, value=0.03321410717293369), FeatureWeight(feature='after', weight=0.004044317147435849, std=None, value=0.05891606103043584), FeatureWeight(feature='its', weight=0.003643504055527284, std=None, value=0.04926549190062908), FeatureWeight(feature='22', weight=0.003445922767757339, std=None, value=0.09025932323927115), FeatureWeight(feature='21', weight=0.002985011983293966, std=None, value=0.10278410163838514), FeatureWeight(feature='power', weight=0.0026705171957969545, std=None, value=0.07494912904527576), FeatureWeight(feature='immediately', weight=0.002231726167876754, std=None, value=0.09355377088368204), FeatureWeight(feature='more', weight=0.001380047483671076, std=None, value=0.05268498804192919), FeatureWeight(feature='confidential', weight=0.0009227543789758396, std=None, value=0.11011062733170919), FeatureWeight(feature='cease', weight=0.000917058970784021, std=None, value=0.11011062733170919), FeatureWeight(feature='cia', weight=7.739177634826758e-05, std=None, value=0.11011062733170919)], neg=[FeatureWeight(feature='of', weight=-0.054336734921217114, std=None, value=0.1512659825099557), FeatureWeight(feature='or', weight=-0.03954753051417578, std=None, value=0.19802091567813376), FeatureWeight(feature='shall', weight=-0.03488856852483867, std=None, value=0.23782194891225913), FeatureWeight(feature='meaning', weight=-0.02375398669216943, std=None, value=0.3317311901837885), FeatureWeight(feature='section', weight=-0.015858480192910075, std=None, value=0.3241159699382722), FeatureWeight(feature='forth', weight=-0.013727380931532758, std=None, value=0.3109381793606286), FeatureWeight(feature='entity', weight=-0.01276241656354817, std=None, value=0.1951716978651904), FeatureWeight(feature='set', weight=-0.012698700488332921, std=None, value=0.2948098671649761), FeatureWeight(feature='such', weight=-0.012464832430617783, std=None, value=0.09915950175400985), FeatureWeight(feature='party', weight=-0.011778037771744316, std=None, value=0.07494912904527576), FeatureWeight(feature='securities', weight=-0.010816694458311085, std=None, value=0.0850610705334812), FeatureWeight(feature='in', weight=-0.010478696851149225, std=None, value=0.09292309724326082), FeatureWeight(feature='have', weight=-0.009332991282196858, std=None, value=0.15625902775199124), FeatureWeight(feature='than', weight=-0.007364674651911239, std=None, value=0.055097824952837736), FeatureWeight(feature='50', weight=-0.007286807180526857, std=None, value=0.1749478148887795), FeatureWeight(feature='result', weight=-0.006418367106859538, std=None, value=0.09355377088368204), FeatureWeight(feature='related', weight=-0.00641036756143805, std=None, value=0.07773454484015715), FeatureWeight(feature='as', weight=-0.0053452489416480585, std=None, value=0.029728446784566896), FeatureWeight(feature='19', weight=-0.004637118299330632, std=None, value=0.10278410163838514), FeatureWeight(feature='hold', weight=-0.004631053904405821, std=None, value=0.09025932323927115), FeatureWeight(feature='prior', weight=-0.004528742736747986, std=None, value=0.07370246679124402), FeatureWeight(feature='transaction', weight=-0.004104397169229397, std=None, value=0.0975858489325952), FeatureWeight(feature='claims', weight=-0.004075382653916751, std=None, value=0.0850610705334812), FeatureWeight(feature='information', weight=-0.004072415709325689, std=None, value=0.09025932323927115), FeatureWeight(feature='manager', weight=-0.0035246909853817597, std=None, value=0.11011062733170919), FeatureWeight(feature='reorganization', weight=-0.0

In [14]:
print(preds_with_probs[highest_degree])
print(filenames[highest_degree])
show_weights(filenames[highest_degree])

0.934672261483211
../../dataset/minNarrative_txtfiles/reddit-stories/REDDIT_5S_bhbyx-143-story.txt


Explanation(estimator='TheilSenRegressor(n_jobs=-1, random_state=0)', description=None, error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=0.3039374326740319, std=None, value=1.0), FeatureWeight(feature='out', weight=0.07394336940513739, std=None, value=0.15878288165890778), FeatureWeight(feature='me', weight=0.07151378555026883, std=None, value=0.17679078085935646), FeatureWeight(feature='night', weight=0.050929401137670756, std=None, value=0.25943008584145677), FeatureWeight(feature='his', weight=0.04845124079879188, std=None, value=0.20910799348802234), FeatureWeight(feature='my', weight=0.04756057208792819, std=None, value=0.16519444124686417), FeatureWeight(feature='us', weight=0.047424719239679426, std=None, value=0.21973151444986522), FeatureWeight(feature='was', weight=0.04141332408435693, std=None, value=0.1096721417784023), FeatureWeight(feature='she', weight=0.03970889245526913, std=None, value=0.17795785538052014), FeatureWeight(feature='think', weight=0.029243412669933428, std=None, value=0.12573461861384877), FeatureWeight(feature='man', weight=0.0273001206927726, std=None, value=0.113469823430177), FeatureWeight(feature='house', weight=0.020876954808664985, std=None, value=0.12391687311448223), FeatureWeight(feature='bed', weight=0.017508419997008293, std=None, value=0.13957855534445177), FeatureWeight(feature='again', weight=0.016584364218128568, std=None, value=0.12391687311448223), FeatureWeight(feature='ass', weight=0.015540604098642753, std=None, value=0.16465298834495054), FeatureWeight(feature='when', weight=0.014671489821581084, std=None, value=0.1513883188945844), FeatureWeight(feature='slept', weight=0.014218547409995065, std=None, value=0.1588548185387044), FeatureWeight(feature='her', weight=0.011494894279138006, std=None, value=0.08616276242854094), FeatureWeight(feature='do', weight=0.010773890694796274, std=None, value=0.08839539042967823), FeatureWeight(feature='creepy', weight=0.010463765391986422, std=None, value=0.19379276396292655), FeatureWeight(feature='at', weight=0.008904721279965012, std=None, value=0.058456243394320255), FeatureWeight(feature='there', weight=0.008618591721083422, std=None, value=0.08407664497008334), FeatureWeight(feature='fucking', weight=0.008032915163533, std=None, value=0.18089820783550192), FeatureWeight(feature='creeped', weight=0.007541107568490006, std=None, value=0.19379276396292655), FeatureWeight(feature='it', weight=0.006740528268152704, std=None, value=0.053074285088283664), FeatureWeight(feature='grabbed', weight=0.006499731321557054, std=None, value=0.19379276396292655), FeatureWeight(feature='slipped', weight=0.0055490037570719115, std=None, value=0.19379276396292655), FeatureWeight(feature='into', weight=0.005361733712626835, std=None, value=0.08458548604031076), FeatureWeight(feature='good', weight=0.005122492161895215, std=None, value=0.11476803994510935), FeatureWeight(feature='country', weight=0.00502480633538246, std=None, value=0.14970598536933152), FeatureWeight(feature='so', weight=0.0045462829805708194, std=None, value=0.08510260131077294), FeatureWeight(feature='tongue', weight=0.003989387827118078, std=None, value=0.18089820783550192), FeatureWeight(feature='and', weight=0.003605698515779492, std=None, value=0.07104646555222427), FeatureWeight(feature='bad', weight=0.003550574387262274, std=None, value=0.142609599048153), FeatureWeight(feature='parents', weight=0.002544448020932104, std=None, value=0.18089820783550192), FeatureWeight(feature='the', weight=0.0022649972000062272, std=None, value=0.03381533846860692)], neg=[FeatureWeight(feature='of', weight=-0.013661670864600597, std=None, value=0.038032209131773055), FeatureWeight(feature='were', weight=-0.010883204513601537, std=None, value=0.07608437156283542), FeatureWeight(feature='gave', weight=-0.007991920489995632, std=None, value=0.13681142924190687), FeatureWeight(feat

In [15]:
eli5.show_weights(model, feature_names=feature_names, top=(51,50))

Weight?,Feature
+0.466,out
+0.405,me
+0.378,was
+0.318,door
+0.304,<BIAS>
+0.295,different
+0.294,woman
+0.288,my
+0.283,plane
+0.268,while
